# Pytorch Lightning for ResNet using galaxy_datasets

## Imports

In [2]:
import os
from enum import Enum
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import lightning as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import albumentations as A
from galaxy_datasets.pytorch.galaxy_datamodule import GalaxyDataModule
from ChiralityClassifier import ChiralityClassifier

GZDESI/GZRings/GZCD not available from galaxy_datasets.pytorch.datasets - skipping


## Options

In [2]:
class modes(Enum):
    FULL_DATASET = 0 #Use all 600,000 galaxies
    CUT_DATASET = 1 #Use cut of 200,000 galaxies
    BEST_SUBSET = 2 #Select N best S,Z & other galaxies, evenly split
    LOCAL_SUBSET = 3 #Use local cache of 1500 galaxies

IMG_SIZE = 160 # This is the output size of the generated image array
MODE = modes.BEST_SUBSET

#If using best subset, Number of CW, ACW and EL to select
THRESHOLD = 0.8
N_CW = 5000
N_ACW = 5000
N_EL = 5000

# FULL_CATALOG_PATH = '../Data/gz1_desi_cross_cat.csv'
# FULL_DATA_PATH = '/share/nas2/walml/galaxy_zoo/decals/dr8/jpg'
# CUT_CATALOG_PATH = '../Data/gz1_desi_cross_cat_cut.csv'
# LOCAL_SUBSET_CATALOG_PATH = '../Data/subset_gz1_desi_cross_cat.csv'
# LOCAL_SUBSET_DATA_PATH = '../Data/Subset'
FULL_CATALOG_PATH = '/share/nas2/npower/mphys-galaxy/Data/gz1_desi_cross_cat.csv'
FULL_DATA_PATH = '/share/nas2/walml/galaxy_zoo/decals/dr8/jpg'
CUT_CATALOG_PATH = '/share/nas2/npower/mphys-galaxy/Data/gz1_desi_cross_cat_cut.csv'
LOCAL_SUBSET_CATALOG_PATH = '/share/nas2/npower/mphys-galaxy/Data/subset_gz1_desi_cross_cat.csv'
LOCAL_SUBSET_DATA_PATH = '/share/nas2/npower/mphys-galaxy/Data/Subset'
SAVE_PATH = "/share/nas2/npower/mphys-galaxy/Models"

torch.set_float32_matmul_precision("medium")

## GPU Test

In [4]:
#Check GPU & Torch is working
print(f"Using pytorch {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"CPU cores available on device: {os.cpu_count()}")
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
print('Using device:', device)

Using pytorch 2.2.1
CPU cores available on device: 24
NVIDIA A100-PCIE-40GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
Using device: cuda


## Reading in data

### Building catalog

In [4]:
def get_file_paths(catalog_to_convert,folder_path ):
    brick_ids = catalog_to_convert['dr8_id'].str.split("_",expand=True)[0]
    dr8_ids = catalog_to_convert['dr8_id']
    file_locations = folder_path+'/'+brick_ids+'/'+dr8_ids+'.jpg'
    print(f"Created {file_locations.shape[0]} galaxy filepaths")
    return file_locations

if MODE == modes.FULL_DATASET:
    catalog = pd.read_csv(FULL_CATALOG_PATH)
    catalog['file_loc'] = get_file_paths(catalog,FULL_DATA_PATH)

elif MODE == modes.CUT_DATASET:
    catalog = pd.read_csv(CUT_CATALOG_PATH)
    catalog['file_loc'] = get_file_paths(catalog,FULL_DATA_PATH)

elif MODE == modes.BEST_SUBSET:
    catalog = pd.read_csv(FULL_CATALOG_PATH)
    very_CW_galaxies = catalog[catalog['P_CW']>THRESHOLD]
    very_ACW_galaxies = catalog[catalog['P_ACW']>THRESHOLD]
    very_EL_galaxies = catalog[catalog['P_EL']>THRESHOLD]
    print(f"Very CW: {very_CW_galaxies.shape[0]}, Very ACW: {very_ACW_galaxies.shape[0]}, Very EL: {very_EL_galaxies.shape[0]}")

    galaxy_subset = pd.concat([very_CW_galaxies[0:N_CW],very_ACW_galaxies[0:N_ACW],very_EL_galaxies[0:N_EL]])
    catalog = galaxy_subset.reset_index()
    catalog['file_loc'] = get_file_paths(catalog,FULL_DATA_PATH)

elif MODE == modes.LOCAL_SUBSET:
    catalog = pd.read_csv(LOCAL_SUBSET_CATALOG_PATH)
    catalog['file_loc'] = get_file_paths(catalog,LOCAL_SUBSET_DATA_PATH)

Very CW: 14243, Very ACW: 15420, Very EL: 143858
Created 15000 galaxy filepaths


### Merging non-S/Z galaxies

In [5]:
catalog['P_OTHER'] = catalog['P_EL']+catalog['P_EDGE']+catalog['P_DK']+catalog['P_MG']
print(f"Loaded {catalog.shape[0]} galaxy images")

Loaded 15000 galaxy images


## Code to run

In [6]:
def generate_transforms(resize_after_crop=IMG_SIZE):
    transforms_to_apply = [
        A.ToFloat(), #Converts from 0-255 to 0-1

        A.Resize( #Resizes to 160x160
            height=resize_after_crop,
            width=resize_after_crop,
            interpolation=1,
            always_apply=True
        ),
    ]

    return A.Compose(transforms_to_apply)

datamodule = GalaxyDataModule(
    label_cols=['P_CW','P_ACW','P_OTHER'],
    catalog=catalog,
    train_fraction=0.7,
    val_fraction=0.15,
    test_fraction=0.15,
    custom_albumentation_transform=generate_transforms(),
    batch_size=200,
    num_workers=11,
)

datamodule.prepare_data()
datamodule.setup()

In [7]:
RUN_TEST = False

model = ChiralityClassifier(
    num_classes=3, #2 for Jia et al version
    model_version="jiaresnet50",
    optimizer="adamw",
    scheduler  ="steplr",
    lr=0.0001,
    weight_decay=0,
    step_size=5,
    gamma=0.85,
    batch_size=60,
)

#stopping_callback = EarlyStopping(monitor="val_loss", mode="min")

trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=60,
    devices=1,
    default_root_dir="/share/nas2/npower/mphys-galaxy/Code/"
    #callbacks=[stopping_callback]
)

#compiled_model = torch.compile(model, backend="eager")
trainer.fit(model,train_dataloaders=datamodule.train_dataloader(),val_dataloaders=datamodule.val_dataloader() )

if RUN_TEST:
    trainer.test(model,test_dataloader=datamodule.test_dataloader())
    
#torch.save(trainer.model.state_dict(), SAVE_PATH + "/trained_model.pt")

/share/nas2/npower/miniconda3/envs/mphys-galaxy/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /share/nas2/npower/miniconda3/envs/mphys-galaxy/lib/ ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type             | Params
---------------------------------------------
0 | loss_fn | CrossEntropyLoss | 0     
1 | model   | JiaResnet        | 24.9 M
---------------------------------------------
24.9 M    Trainable params
0         Non-trainable params
24.9 M    Total params
99.428    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 60.00 MiB. GPU 0 has a total capacity of 39.41 GiB of which 23.56 MiB is free. Process 22038 has 5.86 GiB memory in use. Including non-PyTorch memory, this process has 33.51 GiB memory in use. Of the allocated memory 31.74 GiB is allocated by PyTorch, and 802.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)